In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

In [ ]:
df=pd.read_csv('../input/bluebook-for-bulldozers/TrainAndValid.csv',low_memory=False)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
fig,ax=plt.subplots()
ax.scatter(df['saledate'][:1000],df['SalePrice'][:1000])

In [ ]:
df.SalePrice.plot.hist()

In [ ]:
df=pd.read_csv('../input/bluebook-for-bulldozers/TrainAndValid.csv',parse_dates=['saledate'])

In [ ]:
df.saledate[:100]

In [ ]:
fig,ax=plt.subplots()
ax.scatter(df.saledate[:1000],df.SalePrice[:1000])

In [ ]:
df.head()

In [ ]:
df.head().T

In [ ]:
df.saledate.head(20)

In [ ]:
df.sort_values(by=['saledate'],inplace=True,ascending=True)

In [ ]:
df.saledate.head(20)

In [ ]:
df_temp=df.copy()

In [ ]:
df_temp['saleYear']=df_temp.saledate.dt.year
df_temp['saleMonth']=df_temp.saledate.dt.month
df_temp['saleDay']=df_temp.saledate.dt.day
df_temp['saleDayOfWeek']=df_temp.saledate.dt.dayofweek
df_temp['saleDayOf Year']=df_temp.saledate.dt.dayofyear






In [ ]:
df_temp.drop('saledate',axis=1,inplace=True)

In [ ]:
df_temp.state.value_counts()

In [ ]:
df_temp.info()

In [ ]:
df_temp.isna().sum()

In [ ]:
df_temp.head().T

In [ ]:
pd.api.types.is_string_dtype(df_temp['UsageBand'])

In [ ]:
df_temp.items()

In [ ]:
for i ,j in df_temp.items():
    print(i,j)

In [ ]:
for label,content in df_temp.items():
    if pd.api.types.is_string_dtype(content):
        print(label)

In [ ]:
for label,content in df_temp.items():
    if pd.api.types.is_string_dtype(content):
        df_temp[label]=content.astype('category').cat.as_ordered()

In [ ]:
df_temp.info()

In [ ]:
df_temp.state.cat.categories

In [ ]:
df_temp.state.cat.codes

In [ ]:
df_temp.isnull().sum()/len(df_temp)

In [ ]:
df_temp.to_csv('train_temp.csv',index=False)

In [ ]:
df_temp=pd.read_csv('train_temp.csv',low_memory=False)

In [ ]:
df_temp.head().T

In [ ]:
df_temp.isna().sum()

In [ ]:
for label,content in df_temp.items():
    if pd.api.types.is_numeric_dtype(content):
        print(label)

In [ ]:
for label,content in df_temp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)

In [ ]:
for label,content in df_temp.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            df_temp[label+'is_missing']=pd.isnull(content)
            df_temp[label]=content.fillna(content.median())

In [ ]:
df_temp.head().T

In [ ]:
df_temp.auctioneerIDis_missing.value_counts()

In [ ]:
df_temp.isna().sum()

In [ ]:
pd.Categorical(df_temp['state'])

In [ ]:
df_temp.state

In [ ]:
df_temp.state.value_counts()

In [ ]:
pd.Categorical(df_temp['state']).codes

In [ ]:
for label,content in df_temp.items():
    if not pd.api.types.is_numeric_dtype(content):
        df_temp[label+'missing']=pd.isnull(content)
        df_temp[label]=pd.Categorical(content).codes + 1 
    

In [ ]:
df_temp.info()

In [ ]:
df_temp.head().T

In [ ]:
df_temp.isna().sum()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model=RandomForestRegressor(n_jobs=-1,random_state=42)


In [ ]:
x=df_temp.drop('SalePrice',axis=1)
y=df_temp['SalePrice']


In [ ]:
model.fit(x,y)

In [ ]:
import pickle

In [ ]:
pickle.dump(model,open('Tsmodel1.pkl','wb'))

In [ ]:
model.score(x,y)

In [ ]:
df_val=df_temp[df_temp.saleYear==2012]
df_train=df_temp[df_temp.saleYear!=2012]

In [ ]:
len(df_val), len(df_train)

In [ ]:
X_train,y_train=df_train.drop('SalePrice',axis=1), df_train['SalePrice']

X_val,y_val=df_val.drop('SalePrice',axis=1), df_val['SalePrice']

In [ ]:
from sklearn.metrics import mean_squared_log_error,mean_absolute_error,r2_score

def rmsle(y_test,y_preds):
    return np.sqrt(mean_squared_log_error(y_test,y_preds))
def show_scores(model):
    train_preds=model.predict(X_train)
    val_preds=model.predict(X_val)
    scores={'training mae':mean_absolute_error(y_train,train_preds),
           'valid mae':mean_absolute_error(y_val,val_preds),
           'training rmsle':rmsle(y_train,train_preds),
           'valid rmsle':rmsle(y_val,val_preds),
           'training r2':r2_score(y_train,train_preds),
           'valid r2':r2_score(y_val,val_preds)}
    return scores

In [ ]:
%%time
model=RandomForestRegressor(random_state=42,
                           max_samples=10000)

model.fit(X_train,y_train)

In [ ]:
show_scores(model)

In [ ]:
pickle.dump(model,open('Tsmodel1.pkl','wb'))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rf_grid={'n_estimators':np.arange(10,100,10),
        'max_depth':[None,3,5,10],
        'min_samples_split':np.arange(2,20,2),
        'min_samples_leaf':np.arange(1,20,2),
        'max_features':[0.5,1,'sqrt','auto'],
        'max_samples':[10000]}

rs_model=RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,random_state=42),
                           param_distributions=rf_grid,
                           n_iter=2,
                           cv=5,
                           verbose=True)


In [ ]:
rs_model.fit(X_train,y_train)

In [ ]:
rs_model.best_params_

In [ ]:
show_scores(rs_model)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rf_grid={'n_estimators':np.arange(10,100,10),
        'max_depth':[None,3,5,10],
        'min_samples_split':np.arange(2,20,2),
        'min_samples_leaf':np.arange(1,20,2),
        'max_features':[0.5,1,'sqrt','auto'],
        'max_samples':[10000]}

rs_model=RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,random_state=42),
                           param_distributions=rf_grid,
                           n_iter=100,
                           cv=5,
                           verbose=True)


In [ ]:
rs_model.fit(X_train,y_train)

In [ ]:
rs_model.best_params_

In [ ]:
show_scores(rs_model)

In [ ]:
pickle.dump(model,open('Tsmodel1.pkl','wb'))

In [ ]:
m=pickle.load(open('Tsmodel1.pkl','rb'))

In [ ]:
ideal_model=RandomForestRegressor(n_estimators=40,
                                 min_samples_leaf=1,
                                 min_samples_split=14,
                                 max_features=0.5,
                                 n_jobs=-1,
                                 max_samples=None,
                                 random_state=42)

In [ ]:
ideal_model.fit(X_train,y_train)

In [ ]:
show_scores(ideal_model)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

rf_grid={'n_estimators':np.arange(10,100,10),
        'max_depth':[None,1,3,5,10],
        'min_samples_split':np.arange(2,20,2),
        'min_samples_leaf':np.arange(1,20,2),
        'max_features':[0.5,1,'sqrt','auto'],
        'max_samples':[10000]}

rs_model=RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,random_state=42),
                           param_distributions=rf_grid,
                           n_iter=200,
                           cv=5,
                           verbose=True)


In [ ]:
rs_model.fit(X_train,y_train)

In [ ]:
rs_model.best_params_

In [ ]:
show_scores(rs_model)

In [ ]:


df_data_2 = pd.read_csv('../input/bluebook-for-bulldozers/Test.csv',low_memory=False)
df_test=df_data_2
df_test.head()
df_test.drop('SalePrice',axis=1,inplace=True)


In [ ]:
df_test.isna().sum()

In [ ]:
X_train.columns

In [ ]:
def preprocess_data(df): 
    df['saleYear']=df.saledate.dt.year
    df['saleMonth']=df.saledate.dt.month
    df['saleDay']=df.saledate.dt.day
    df['saleDayOfWeek']=df.saledate.dt.dayofweek
    df['saleDayOf Year']=df.saledate.dt.dayofyear
    
    df.drop('saledate',axis=1,inplace=True)
    
    for label,content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                df[label+'is_missing']=pd.isnull(content)
                df[label]=content.fillna(content.median())
                
        
        if not pd.api.types.is_numeric_dtype(content):
            df[label+'missing']=pd.isnull(content)
            df[label]=pd.Categorical(content).codes + 1 
    
    
    return df

In [ ]:
df_test=preprocess_data(df_test)

In [ ]:
df_test.head()

In [ ]:
test_preds=ideal_model.predict(df_test)

In [ ]:
X_train.head()

In [ ]:
set(X_train.columns) - set(df_test.columns)

In [ ]:
#df_test['auctioneerID_is_missing']=False

In [ ]:
test_preds=ideal_model.predict(df_test)

In [ ]:
test_preds

In [ ]:
ideal_model.feature_importances_

In [ ]:
def plot_features(columns,importances,n=20):
    df=(pd.DataFrame({'features':columns,'feature_importances':importances}).sort_values('feature_importances',ascending=False).reset_index(drop=True))
    fig,ax=plt.subplots()
    ax.barh(df['features'][:n],df['feature_importances'][:20])
    ax.set_xlabel('features_importances')
    ax.set_ylabel('feature')
    ax.invert_yaxis()
    

In [ ]:
plot_features(X_train.columns,ideal_model.feature_importances_)

In [ ]:

filename = 'finalized_model.sav'
pickle.dump(ideal_model, open(filename, 'wb'))